In [105]:
import tensorflow as tf
import tensorflow.compat.v1 as v1
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
import math

![](1.png)

In [106]:
def write_to_submission_file(predict, out_file, label = 'ImageId'):
    label = np.arange(1, predict.shape[0]+1)
    predicted_df = pd.DataFrame({'ImageId':label, 'Label':predict})
    predicted_df.to_csv(out_file)
    return predicted_df

In [107]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
samples = pd.read_csv('sample_submission.csv')

In [108]:
samples

,ImageId,Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
27995,27996,0
27996,27997,0
27997,27998,0
27998,27999,0


In [109]:
train_data['label']

0        1
1        0
2        1
3        4
4        0
        ..
41995    0
41996    1
41997    7
41998    6
41999    9
Name: label, Length: 42000, dtype: int64

In [110]:
train_data.describe()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
count,42000.000000,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,...,42000.000000,42000.000000,42000.000000,42000.00000,42000.000000,42000.000000,42000.0,42000.0,42000.0,42000.0
mean,4.456643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.219286,0.117095,0.059024,0.02019,0.017238,0.002857,0.0,0.0,0.0,0.0
std,2.887730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.312890,4.633819,3.274488,1.75987,1.894498,0.414264,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.00000,254.000000,62.000000,0.0,0.0,0.0,0.0


In [111]:
train_data.shape, test_data.shape

((42000, 785), (28000, 784))

In [112]:
from sklearn.preprocessing import OneHotEncoder
model = OneHotEncoder()
label = model.fit_transform(np.array(train_data['label']).reshape(-1, 1)).toarray()
label

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [113]:
X_tain = np.array(train_data.drop(['label'], axis = 1)/255).reshape(-1, 28,28, 1)

In [114]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (1 for blue dot / 0 for red dot), of shape (1, number of examples)
    mini_batch_size -- size of the mini-batches, integer
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    np.random.seed(seed)            # To make your "random" minibatches the same as ours
    m = X.shape[0]                  # number of training examples
    mini_batches = []
        
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation,:]
    shuffled_Y = Y[permutation,:]

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k*mini_batch_size:(k+1)*mini_batch_size, :]
        mini_batch_Y = shuffled_Y[k*mini_batch_size:(k+1)*mini_batch_size, :]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[mini_batch_size*num_complete_minibatches:, :]
        mini_batch_Y = shuffled_Y[mini_batch_size*num_complete_minibatches:, :]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [115]:
X_test = (np.array(test_data)/255).reshape(-1, 28,28, 1)

![LeNET -5](LeNEt_5.png)

In [116]:
#Tensorflow
tf.compat.v1.disable_eager_execution()

In [117]:
x = v1.placeholder(tf.float32, shape = [None, 28,28,1], name = 'x')               #m, 28 28 ,1
y = v1.placeholder(tf.float32, shape = [None, 10], name = 'y')

In [118]:
W1 = v1.Variable(v1.truncated_normal([5,5,1,6], mean=0.0, stddev=0.1))
b1 = v1.Variable(v1.constant(0.1, shape = [6]))

In [119]:
h_conv_1 = tf.nn.tanh(tf.nn.conv2d(x, W1, strides = [1,1,1,1], padding = 'VALID') + b1) #m, 24 24 ,6

In [120]:
h_pool_2 = tf.nn.tanh(tf.nn.avg_pool(h_conv_1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID'))         # m 12 12 6

In [121]:
W2 = v1.Variable(v1.truncated_normal([5,5,6,16], mean=0.0, stddev=0.1))
b2 = v1.Variable(v1.constant(0.1, shape = [16]))

In [122]:
h_conv_3 = tf.nn.tanh((tf.nn.conv2d(h_pool_2, W2, strides = [1,1,1,1], padding = 'VALID') + b2))# m 8 8 16

In [123]:
h_pool_4 = tf.nn.tanh(tf.nn.avg_pool(h_conv_3, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID'))         # m 4 4 16

In [124]:
W3 = v1.Variable(v1.truncated_normal([4*4*16, 120], mean=0.0, stddev=0.1))
b3 = v1.Variable(v1.constant(0.1, shape = [120]))

In [125]:
h_flat_5 = tf.reshape(h_pool_4, [-1, 4*4*16])

In [126]:
h_fc_6 = tf.nn.tanh(v1.matmul(h_flat_5, W3) +b3)

In [127]:
W4 = v1.Variable(v1.truncated_normal([120, 84], mean=0.0, stddev=0.1))
b4 = v1.Variable(v1.constant(0.1, shape = [84]))

In [128]:
h_fc_7 = tf.nn.tanh(v1.matmul(h_fc_6, W4) +b4)

In [129]:
W5 = v1.Variable(v1.truncated_normal([84, 10], mean=0.0, stddev=0.1))
b5 = v1.Variable(v1.constant(0.1, shape = [10]))

In [130]:
logit_conv = v1.matmul(h_fc_7, W5) +b5

In [131]:
h_fc_8 = tf.nn.softmax(logit_conv)

In [132]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(h_fc_8, y))

In [133]:
train_step = v1.train.AdamOptimizer(0.0001).minimize(cross_entropy)

In [134]:
init = v1.global_variables_initializer()
sess = v1.Session()
sess.run(init)

In [135]:
for i in tqdm(range(100)):
    minibatches = random_mini_batches(X_tain, label)
    for minibatch in minibatches:
        (minibatch_X, minibatch_Y) = minibatch
        sess.run(train_step, feed_dict = {x:minibatch_X.reshape(-1,28,28,1), y:minibatch_Y.reshape(-1,10)})

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [12:40<00:00,  7.60s/it]


In [136]:
predictions = tf.argmax(h_fc_8, 1)
predicted_labels = predictions.eval(feed_dict={x: X_test}, session=sess)
predicted_labels

array([2, 0, 9, ..., 3, 9, 2], dtype=int64)

In [137]:
sess.close()

In [143]:
#write_to_submission_file(predicted_labels, 'out_file.csv', label = 'ImageId')

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [146]:
output = pd.DataFrame({'ImageId': test_data.index +1, 'Label': predicted_labels})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")
output                                                             # 0.98557

Your submission was successfully saved!


,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


## Keras

In [149]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

In [150]:
batch_size, h_img, w_img = 64, 28, 28
input_shape = (h_img, w_img, 1)

In [151]:
model = Sequential()